In [1]:
from collections import defaultdict
from functools import partial
from pathlib import Path
import shutil
import sys
import time
from typing import Any, Dict, Optional, Tuple, OrderedDict
import random
import hydra
from hydra.utils import instantiate
from hydra.utils import get_original_cwd, to_absolute_path
from omegaconf import DictConfig, OmegaConf, open_dict
from torch.utils.tensorboard import SummaryWriter
from collections import deque
from tqdm import tqdm
import wandb
import ray

import envpool
# import gymnasium as gym
import gym
from gym.spaces import Box, Discrete
from gym.spaces.dict import Dict as GymDict
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
from modules.networks.blocks import S4Layer

In [2]:
import torch
# from src.models.sequence.modules.s4block import S4Block
from models.s4.s4 import S4Block
s4_block = S4Block(
        d_model=32,
        bottleneck=None,
        gate=None,
        gate_act="id",
        mult_act=None,
        final_act="tanh",
        postact=None,
        initializer='uniform',
        weight_norm=False,
        dropout=0.1,
        tie_dropout=False,
        transposed=False,
                        )

In [1]:
from modules.networks.s4 import S4Model

[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


In [5]:
model = S4Model(
    d_input=1,
    # d_output=32,
    d_model=64,
    n_layers=1,
    dropout=0.1,
    # prenorm=args.prenorm,
).to('cuda:4')

In [6]:
total_params = 0
for name, p in model.named_parameters():
    total_params += p.numel()
total_params

16896

In [5]:
a = torch.ones([55, 17, 1])
b = model(a)
print(b.shape)

torch.Size([55, 17, 32])


In [6]:
cfg_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DARL_transformer/configs/ppo_trainer.yaml"
nn_cfg_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DARL_transformer/configs/nn/nn.yaml"
ppo_cfg_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DARL_transformer/configs/ppo/ppo.yaml"

In [7]:
cfg = OmegaConf.load(cfg_path)
nn_cfg = OmegaConf.load(nn_cfg_path)
ppo_cfg = OmegaConf.load(ppo_cfg_path)
cfg.nn = nn_cfg
cfg.ppo = ppo_cfg
cfg

{'defaults': [{'ppo': 'ppo'}, {'nn': 'nn'}, '_self_'], 'hydra': {'run': {'dir': 'outputs/${now:%Y-%m-%d/%H-%M-%S}'}}, 'experiment': {'env_ids': ['HalfCheetah-v4'], 'seed': 42, 'max_episode_steps': 1000, 'num_rollout_steps': 128, 'num_train_envs': 64, 'num_envs': 64, 'total_timesteps': 10000000, 'save_ckpt': False, 'num_checkpoints': 20, 'print_interval': 100, 'stop_after_epochs': 500, 'capture_video': False, 'device': 0, 'cuda': True, 'torch_deterministic': True, 'resume': False, 'resume_update_idx': 0, 'resume_dir': 'None'}, 'evaluation': {'eval_seed': 3142, 'every': 8, 'num_eval': 5, 'num_test_envs': 5}, 'wandb': {'mode': 'online', 'project': 'DomainAgnosticRL', 'entity': None, 'name': None, 'group': None, 'tags': None, 'notes': None}, 'paths': {'dir': 'outputs/${now:%Y-%m-%d/%H-%M-%S}', 'log': 'outputs/${now:%Y-%m-%d/%H-%M-%S}/runs', 'video': 'outputs/${now:%Y-%m-%d/%H-%M-%S}/videos', 'checkpoints': 'outputs/${now:%Y-%m-%d/%H-%M-%S}/checkpoints', 'src': 'outputs/${now:%Y-%m-%d/%H-%M

In [8]:
from modules.networks.weight_generator import S4EncoderTransformer, S4DecoderTransformer
from modules.networks.weight_generator import RNNEncoderTransformer, RNNDecoderTransformer




In [9]:
s4_encoder_transformer = S4EncoderTransformer(cfg)
s4_decoder_transformer = S4DecoderTransformer(cfg)
rnn_encoder_transformer = RNNEncoderTransformer(cfg)
rnn_decoder_transformer = RNNDecoderTransformer(cfg)


In [10]:
obs = torch.ones([32, 17])
out, _ = rnn_encoder_transformer(obs)
out.shape


torch.Size([32, 17, 128])

In [11]:
out, _ = s4_encoder_transformer(obs)
print(out.shape)

torch.Size([32, 17, 128])
